In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error

from scripts.processing import clean_data, pre_process_data
from scripts.pipeline import create_preprocessing_pipeline

In [ ]:
raw_train_data = pd.read_csv('data/train.csv') 

In [ ]:
train_data = clean_data(raw_train_data)

In [ ]:
train_data = pre_process_data(train_data)

In [ ]:
# Passing through sklearn pipeline

num_cols = train_data.select_dtypes(include=['float64', 'int64']).columns.drop('Incurred')
ohe_cols = ['Notifier', 'Location_of_incident', 'Weather_conditions']

pipeline = create_preprocessing_pipeline(num_cols, ohe_cols)

X = train_data.drop(columns=['Incurred'])
y = train_data['Incurred']

X_preprocessed = pipeline.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

In [ ]:
# Comparing different models

models = [LinearRegression(), Ridge(), Lasso(), RandomForestRegressor(), GradientBoostingRegressor()]

results = {}

for model in models:

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[model.__class__.__name__] = root_mean_squared_error(y_test, y_pred)

results['Predicting Mean'] = root_mean_squared_error(y_test, [y_train.mean()]*len(y_test))
results['Predicting Median'] = root_mean_squared_error(y_test, [y_train.median()]*len(y_test))

pd.DataFrame.from_dict(results, orient='index', columns=['RMSE']).sort_values(by='RMSE')

In [ ]:
# Tuning lasso

param_grid = {'alpha': [0.1, 1, 10, 100]}

lasso = Lasso()
lasso_gs = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
lasso_gs.fit(X_train, y_train)
best_lasso = lasso_gs.best_estimator_
y_pred = best_lasso.predict(X_test)
root_mean_squared_error(y_test, y_pred)

In [ ]:
# Plot y_test histogram vs y pred histogram
plt.hist(y_test, bins=100, label='y_test')
plt.hist(y_pred, bins=100, label='y_pred')
plt.xlabel('Incurred')
plt.ylabel('Count')
plt.legend()

In [ ]:
# Plot feature importance graph

feature_names = X_preprocessed.columns
coefficients = best_lasso.coef_

coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

coef_df['AbsCoefficient'] = coef_df['Coefficient'].abs()
coef_df = coef_df.sort_values(by='AbsCoefficient', ascending=False)

# Plotting
plt.figure(figsize=(10, 6))
bars = plt.barh(coef_df['Feature'], coef_df['Coefficient'], color=np.where(coef_df['Coefficient'] > 0, 'green', 'red'))

# Add labels and title
plt.ylabel('Feature')
plt.xlabel('Coefficient')
plt.title('Feature Importance in Lasso Regression')
plt.tight_layout()